In [ ]:
from urllib.request import urlopen
import timm
import torch
import zipfile,os
from PIL import Image
from pathlib import Path

local_zip = '/DATA/ai lab/gcdData/GCD.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/DATA/ai lab')
zip_ref.close()

['1_cumulus',
 '2_altocumulus',
 '3_cirrus',
 '4_clearsky',
 '5_stratocumulus',
 '6_cumulonimbus',
 '7_mixed']

In [19]:
train_dir = '/DATA/ai lab/GCD/train'
test_dir = '/DATA/ai lab/GCD/test'

os.listdir('/DATA/ai lab/GCD/train')
print(train_dir)

/DATA/ai lab/GCD/train


In [ ]:
model = timm.create_model('maxvit_tiny_tf_224.in1k', pretrained=True,features_only=True)
model = model.eval()

# get model specific transforms (normalization, resize)
data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)

train_dir = Path(train_dir)

for img_path in train_dir.rglob("*.*"):  # Recursively find all files
    if img_path.suffix.lower() in [".jpg", ".jpeg", ".png"]:  # Check for valid image extensions
        img = Image.open(img_path).convert("RGB")  # Open the image and convert to RGB
        img_tensor = transforms(img).unsqueeze(0)  # Preprocess and add batch dimension

        # Pass the image through the model
        output = model(img_tensor)

        # Get top 5 probabilities and class indices
        if isinstance(output, list):  # Handle feature extraction
            output = output[-1]  # Choose the last layer if features_only=True
        top5_probabilities, top5_class_indices = torch.topk(output.softmax(dim=1), k=5)

        # Print the results
        print(f"Image: {img_path}")
        print(f"Top 5 Probabilities: {top5_probabilities}")
        print(f"Top 5 Class Indices: {top5_class_indices}")


AttributeError: 'str' object has no attribute 'rglob'